# Student Performance Prediction using Supervised Learning

## Predict a student’s exam performance based on:
- Study habits
- Demographics
- Family background

This is a regression problem (scores) and can also be converted into classification (Pass / Fail).

### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np

### 2. Load Dataset

In [2]:
df = pd.read_csv("StudentsPerformance.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


### 3. Basic EDA

In [3]:
df.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
math score                      int64
reading score                   int64
writing score                   int64
dtype: object

In [4]:
df.describe()

,math score,reading score,writing score
count,1000.000000,1000.000000,1000.000000
mean,66.396000,69.002000,67.738000
std,15.402871,14.737272,15.600985
min,13.000000,27.000000,23.000000
25%,56.000000,60.000000,58.000000
50%,66.500000,70.000000,68.000000
75%,77.000000,79.000000,79.000000
max,100.000000,100.000000,100.000000


#### 3.1. Separate Features & Target

In [5]:
X = df.drop("math score", axis=1)
y = df["math score"]

### * Preprocessing (Categorical Encoding)

#### 3.2. Manual Encoding 
- Binary Encoding(Manual)
- One-hot Encoding

In [6]:
# X.gender.unique()
# X['race/ethnicity'].unique()
# X['parental level of education'].unique()
# X['lunch'].unique()
# X['test preparation course'].unique()

X["gender"] = X["gender"].map({"male": 0, "female": 1})
X["lunch"] = X["lunch"].map({"free/reduced": 0, "standard": 1})
X["test preparation course"] = X["test preparation course"].map(
    {"none": 0, "completed": 1}
)

One-hot Encoding....

In [7]:
X = pd.get_dummies(X, drop_first=True)

In [8]:
X.head()

,gender,lunch,test preparation course,reading score,writing score,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school
0,0,1,1,67,63,False,False,False,False,False,True,False,False,False
1,1,0,0,59,55,False,False,True,False,False,False,False,False,True
2,0,0,0,60,50,False,False,False,True,False,False,False,True,False
3,0,1,0,78,68,True,False,False,False,False,True,False,False,False
4,0,1,1,73,68,False,False,False,True,False,False,False,False,False


### 4. Train-Test Split

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### 5. Train Model(Linear Regression)

In [10]:
# Convert to NumPy:

X_train_np = X_train.values
X_test_np = X_test.values
y_train_np = y_train.values.reshape(-1, 1)
y_test_np = y_test.values.reshape(-1, 1)


In [11]:
### initialize parameters
n_samples, n_features = X_train_np.shape

W = np.zeros((n_features, 1))  # weights
b = 0                          # bias


In [12]:
### prediction function
def predict(X, W, b):
    return np.dot(X, W) + b


In [13]:
### Loss Function (Mean Squared Error)
def compute_loss(y, y_pred):
    return np.mean((y - y_pred) ** 2)

In [14]:
### Gradent Descent
def gradient_descent(X, y, W, b, lr, epochs):
    n = X.shape[0]

    for i in range(epochs):
        y_pred = predict(X, W, b)

        dW = (2 / n) * np.dot(X.T, (y_pred - y))
        db = (2 / n) * np.sum(y_pred - y)

        W = W - lr * dW
        b = b - lr * db

        if i % 100 == 0:
            loss = compute_loss(y, y_pred)
            print(f"Epoch {i}, Loss: {loss:.4f}")
        if np.isnan(y_pred).any() or np.isinf(y_pred).any():
            print("Divergence detected")
            break


    return W, b


In [15]:
### Manual Scaling Code
import numpy as np

# 1. Select numeric columns ONLY
numeric_cols = ['reading score', 'writing score']

X_train_num = X_train[numeric_cols].values.astype(float)
X_test_num  = X_test[numeric_cols].values.astype(float)

# 2. Compute mean and standard deviation (TRAINING ONLY)
mean = np.mean(X_train_num, axis=0)
std  = np.std(X_train_num, axis=0)

# Avoid division by zero
std[std == 0] = 1

# 3. Apply standard scaling
X_train_scaled = (X_train_num - mean) / std
X_test_scaled  = (X_test_num  - mean) / std

# 4. (Optional) Verify scaling
print("Train mean:", X_train_scaled.mean(axis=0))
print("Train std :", X_train_scaled.std(axis=0))


Train mean: [2.33146835e-16 1.15463195e-16]
Train std : [1. 1.]


In [16]:
# reinitialzizing the weights
W = np.zeros((X_train_scaled.shape[1], 1))
b = 0


In [17]:
### Train the Model
# learning_rate = 0.0001
# epochs = 66000

# W, b = gradient_descent(
#     X_train_np,
#     y_train_np,
#     W,
#     b,
#     learning_rate,
#     epochs
# )

# Train again
W, b = gradient_descent(
    X_train_scaled,
    y_train_np,
    W,
    b,
    lr=0.01,
    epochs=66000
)



Epoch 0, Loss: 4623.1862
Epoch 100, Loss: 153.9167
Epoch 200, Loss: 78.0226
Epoch 300, Loss: 76.6385
Epoch 400, Loss: 76.5725
Epoch 500, Loss: 76.5367
Epoch 600, Loss: 76.5074
Epoch 700, Loss: 76.4830
Epoch 800, Loss: 76.4628
Epoch 900, Loss: 76.4460
Epoch 1000, Loss: 76.4321
Epoch 1100, Loss: 76.4206
Epoch 1200, Loss: 76.4110
Epoch 1300, Loss: 76.4030
Epoch 1400, Loss: 76.3964
Epoch 1500, Loss: 76.3909
Epoch 1600, Loss: 76.3863
Epoch 1700, Loss: 76.3826
Epoch 1800, Loss: 76.3794
Epoch 1900, Loss: 76.3768
Epoch 2000, Loss: 76.3746
Epoch 2100, Loss: 76.3728
Epoch 2200, Loss: 76.3714
Epoch 2300, Loss: 76.3701
Epoch 2400, Loss: 76.3691
Epoch 2500, Loss: 76.3682
Epoch 2600, Loss: 76.3675
Epoch 2700, Loss: 76.3669
Epoch 2800, Loss: 76.3665
Epoch 2900, Loss: 76.3661
Epoch 3000, Loss: 76.3657
Epoch 3100, Loss: 76.3654
Epoch 3200, Loss: 76.3652
Epoch 3300, Loss: 76.3650
Epoch 3400, Loss: 76.3649
Epoch 3500, Loss: 76.3647
Epoch 3600, Loss: 76.3646
Epoch 3700, Loss: 76.3645
Epoch 3800, Loss: 76.

### 7. Prediction

In [18]:
# y_pred_test = predict(X_test_np, W, b)
y_pred_test = predict(X_test_scaled, W, b)

### 8. Evaluation

In [19]:
### manual R2 calculation

# y_test_np : actual values
# y_pred    : predicted values from your model

y_mean = np.mean(y_test_np)

SS_res = np.sum((y_test_np - y_pred_test) ** 2)
SS_tot = np.sum((y_test_np - y_mean) ** 2)

mae = np.mean(np.abs(y_test_np - y_pred_test))
rmse = np.sqrt(np.mean((y_test_np - y_pred_test) ** 2))
r2 = 1 - (SS_res / SS_tot)


print("MAE:", mae)
print("RMSE:", rmse)
print("R2 Score:", r2)

MAE: 6.847407687698157
RMSE: 8.748696394233901
R2 Score: 0.672988053490292
